In [2]:
%cd ../

In [2]:
import torch
torch.backends.cuda.matmul.allow_tf32 = False
import random
import numpy as np
from GMN.configure import *
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

av = Namespace(   want_cuda                    = True,
                  has_cuda                   = torch.cuda.is_available(),
                  DIR_PATH                   =".",
                  special_mode               = "",
                  DATASET_NAME               = "ptc_mm_500qgrlarge",
                  FEAT_TYPE                  = "One",
                  filters_1                  = 10,
                  filters_2                  = 10,
                  filters_3                  = 10,
                  transform_dim              = 16,
                  bottle_neck_neurons        = 10,
                  tensor_neurons             = 10,               
                  dropout                    = 0,
                  BATCH_SIZE                 =128,
                  n_layers                   = 3,
                  conv_type                  = "SAGE",
                  method_type                = "order",
                  skip                       = "learnable",
                  neuromatch_hidden_dim      = 10,
                  post_mp_dim                = 64,
                  bins                       = 16,
                  histogram                  = False,
                  GMN_NPROPLAYERS            = 5,
                  GOSSIP_TEMP                = 1.0,
                  CONV                       = "GCN",
                  gt_mode                    = "qap",
                  mcs_mode                   = "edge",
                  training_mode              = "mse",
                  MARGIN                     = 0.1,
                  NOISE_FACTOR               = 0,
                  TEMP                       = 0.1,
                  NITER                      = 20,
               TASK = ""
              )

def load_config(): 
  # Print configure
  config = get_default_config()
  config['encoder'] ['node_hidden_sizes'] = [av.filters_3]#[10]
  config['encoder'] ['node_feature_dim'] = 1
  config['encoder'] ['edge_feature_dim'] = 1
  config['aggregator'] ['node_hidden_sizes'] = [av.filters_3]#[10]
  config['aggregator'] ['graph_transform_sizes'] = [av.filters_3]#[10]
  config['aggregator'] ['input_size'] = [av.filters_3]#[10]
  config['graph_matching_net'] ['node_state_dim'] = av.filters_3#10
  #config['graph_matching_net'] ['n_prop_layers'] = av.GMN_NPROPLAYERS
  config['graph_matching_net'] ['edge_hidden_sizes'] = [2*av.filters_3]#[20]
  config['graph_matching_net'] ['node_hidden_sizes'] = [av.filters_3]#[10]
  config['graph_matching_net'] ['n_prop_layers'] = 5
  config['graph_embedding_net'] ['node_state_dim'] = av.filters_3#10
  #config['graph_embedding_net'] ['n_prop_layers'] = av.GMN_NPROPLAYERS
  config['graph_embedding_net'] ['edge_hidden_sizes'] = [2*av.filters_3]#[20]
  config['graph_embedding_net'] ['node_hidden_sizes'] = [av.filters_3]#[10]    
  config['graph_embedding_net'] ['n_prop_layers'] = 5

  #logger.info("av gmn_prop_param")
  #logger.info(av.GMN_NPROPLAYERS) 
  #logger.info("config param")
  #logger.info(config['graph_embedding_net'] ['n_prop_layers'] )
  config['graph_embedding_net'] ['n_prop_layers'] = av.GMN_NPROPLAYERS
  config['graph_matching_net'] ['n_prop_layers'] = av.GMN_NPROPLAYERS
  #logger.info("config param")
  #logger.info(config['graph_embedding_net'] ['n_prop_layers'] )

  config['training']['batch_size']  = av.BATCH_SIZE
  config['training']['margin']  = av.MARGIN
  config['evaluation']['batch_size']  = av.BATCH_SIZE
  config['model_type']  = "embedding"
  config['graphsim'] = {} 
  config['graphsim']['conv_kernel_size'] = [10,4,2]
  config['graphsim']['linear_size'] = [24, 16]
  config['graphsim']['gcn_size'] = [10,10,10]
  config['graphsim']['conv_pool_size'] = [3,3,2]
  config['graphsim']['conv_out_channels'] = [2,4,8]
  config['graphsim']['dropout'] = av.dropout 

  # Set random seeds
  seed = config['seed']
  random.seed(seed)
  np.random.seed(seed + 1)
  torch.manual_seed(seed + 2)
  torch.backends.cudnn.deterministic = False
  return config
 
config = load_config()

seed = config['seed']
random.seed(seed)
np.random.seed(seed + 1)
torch.manual_seed(seed + 2)
torch.backends.cudnn.deterministic = False
   

In [3]:
from subgraph.earlystopping import EarlyStoppingModule
from mcs.models import *
from mcs.evaluate import *
es = EarlyStoppingModule(av,50)
device = "cuda" if av.has_cuda and av.want_cuda else "cpu"

Using backend: pytorch


In [4]:
def evaluate_returnAll(av,model,sampler): 
  model.eval() 
 
  pred = [] 
  targets = [] 
 
  n_batches = sampler.create_batches(shuffle=False) 
  for i in range(n_batches): 
    batch_data,batch_data_sizes,target,batch_adj = sampler.fetch_batched_data_by_id(i) 
    pred.append( model(batch_data,batch_data_sizes,batch_adj).data) 
    targets.append(target) 
  all_pred = torch.cat(pred,dim=0).detach() 
  all_target = torch.cat(targets,dim=0).detach() 
  ndcg = ndcg_score([all_target.cpu().tolist()],[all_pred.cpu().tolist()]) 
  mse = torch.nn.functional.mse_loss(all_target, all_pred,reduction="mean").item() 
  mae = torch.nn.functional.l1_loss(all_target, all_pred,reduction="mean").item() 
  rankcorr = kendalltau(all_pred.cpu().tolist(),all_target.cpu().tolist())[0] 
  pairReward = all_pair_similarity_reward1(all_pred,\
                all_target,set(all_target.tolist())).item()
 
  #TODO: query wise stuff 
  return ndcg,mse,rankcorr,mae,pairReward,all_pred,all_target


def pairwse_reward_similarity(scorePos, scoreNeg):
    """
      score* are similarity measures
      We reward +1 for every positive score > some negative score
    """
    scorePosExp = scorePos.unsqueeze(1)
    scoreNegExp = scoreNeg.unsqueeze(1)
    n_1 = scorePosExp.shape[0]
    n_2 = scoreNegExp.shape[0]
    dim = scorePosExp.shape[1]

    expanded_pos = scorePosExp.unsqueeze(1).expand(n_1, n_2, dim)
    expanded_neg = scoreNegExp.unsqueeze(0).expand(n_1, n_2, dim)
    ell = torch.sign(expanded_pos - expanded_neg)
    hinge = torch.nn.ReLU()
    loss = hinge(ell)
    sum_loss =  torch.sum(loss,dim= [0, 1])
    return sum_loss,(n_1*n_2)

def all_pair_similarity_reward1(score_list,target,categories):
    """
      score_list is converted into a 
      list of lists: [list1,list2,list3,..]
      each of the lists can be of any length 
      target is a list of same length as score_list
      each entry of target is a single value denoting importance
      categories are the set of all possible target
      length of categories = length of [list1,list2,list3,..]
      Ex. : score_list = [list1, list2, list3]
            categories = [0.5,0,1]
            here list1 are scores for items with importance 0.5
            here list2 are scores for items with importance 0
            here list3 are scores for items with importance 1
      Return: 
          For each list pair, we reward +1 every time some item in 
          list with greater importance has higher score than some item 
          in a list with lower importance. This is normalized by the 
          max possible reward that can be awarded based on 
          such pairwise interactions. 
    """   
    all_labels = categories
    all_labels = sorted(all_labels)[::-1]
    #assert(all_labels[0]==1)
    all_score_lists = []
    for val in all_labels :
        all_score_lists.append(score_list[target==val])
    total_reward = 0 
    total_max_reward = 0
    for (idx1,idx2) in itertools.combinations(range(len(all_labels)),2):
        pos_list = all_score_lists[idx1]
        neg_list = all_score_lists[idx2]
        if len(pos_list)>0 and len(neg_list)>0:
            reward,max_reward = pairwse_reward_similarity(pos_list,neg_list)
            total_reward+=reward
            total_max_reward += max_reward
    if total_max_reward ==0 : 
        rval = torch.tensor(0)
    else:
        rval = (total_reward/total_max_reward)
    return rval
  



In [11]:
def get_result(task,dataset):
    config = load_config()
    av.TASK = task
    av.DATASET_NAME = dataset 
    av.dataset_stats = pickle.load(open('Datasets/mcs/splits/stats/%s_dataset_stats.pkl' % av.DATASET_NAME, "rb"))

    #if "qap" in av.gt_mode:
    av.TASK = av.TASK + "_gt_mode_" + av.gt_mode
    #if av.training_mode == "rank":
    av.TASK = av.TASK + "_trMode_" + av.training_mode
    if av.FEAT_TYPE == "Adjrow" or  av.FEAT_TYPE == "Adjrow1" or av.FEAT_TYPE == "AdjOnehot": 
        av.TASK = av.TASK + "_" + av.FEAT_TYPE
    if av.CONV != "GCN": 
        av.TASK = av.TASK + "_" + av.CONV
    

    if av.TASK.startswith("ISONET_for_mcs"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_edges']
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = ISONET_for_mcs(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"  
    elif av.TASK.startswith("ISONET_baseline"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_edges']
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = ISONET_baseline(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"  
    elif av.TASK.startswith("T3_ISONET_for_mcs"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_nodes'] 
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = T3_ISONET_for_mcs(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("AllLayersPos_T3_ISONET_for_mcs"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_nodes'] 
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = AllLayersPos_T3_ISONET_for_mcs(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("AsymCrossSinkhorn_T3_ISONET_for_mcs"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_nodes'] 
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = AsymCrossSinkhorn_T3_ISONET_for_mcs(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_nodes'] 
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("T3_GMN_embed"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_edges']
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")    
        model = T3_GMN_embed(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("T3_GMN_match"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_edges']
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")    
        model = T3_GMN_match(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("T5_GMN_embed_nomin"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_edges']
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")    
        model = T5_GMN_embed_nomin(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("T5_GMN_match_nomin"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_edges']
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")    
        model = T5_GMN_match_nomin(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("IsoNetVar29ForMcs"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_nodes'] 
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = IsoNetVar29ForMcs(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_nodes'] 
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("Combo_late_models"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_nodes'] 
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        m1 = Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold(av,config,1).to(device)
        m2 = AllLayersPos_T3_ISONET_for_mcs(av,config,1).to(device)
        model = Combo_late_models(av,config,1,m1,m2).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("Try2Abaltion_NoThresh_IsoNetGossipVar29ForMcs_GossipVector"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_nodes'] 
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = Try2Abaltion_NoThresh_IsoNetGossipVar29ForMcs_GossipVector(av,config,1).to(device)
        test_data.data_type = "gmn"
        val_data.data_type = "gmn"
    elif av.TASK.startswith("SimGNN_for_mcs") :
        av.MAX_SET_SIZE = av.dataset_stats['max_num_edges']
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")    
        model = SimGNN_for_mcs(av,1).to(device)
        test_data.data_type = "pyg"
        val_data.data_type = "pyg"
    elif av.TASK.startswith("GraphSim_for_mcs"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_edges']
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")    
        model = GraphSim_for_mcs(av,config,1).to(device)
        test_data.data_type = "pyg"
        val_data.data_type = "pyg"
    elif av.TASK.startswith("GOTSim_for_mcs"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_nodes'] 
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")
        model = GOTSim_for_mcs(av,config,1).to(device)
        test_data.data_type = "pyg"
        val_data.data_type = "pyg" 
    elif av.TASK.startswith("NeuroMatch"):
        av.MAX_SET_SIZE = av.dataset_stats['max_num_edges']
        test_data = McsData(av,mode="test")
        val_data = McsData(av,mode="val")    
        model = NeuroMatch(1,av.neuromatch_hidden_dim,av).to(device) 
        test_data.data_type = "pyg" 
        val_data.data_type = "pyg" 

    else:
        raise NotImplementedError()    
    model.eval()
    checkpoint = es.load_best_model()
    model.load_state_dict(checkpoint['model_state_dict'])
    

    val_result = evaluate_returnAll(av,model,val_data)
    test_result = evaluate_returnAll(av,model,test_data)
    return val_result, test_result

In [6]:
#QAP,GOSSIP,GLASGOW - MSE,RANK -> 6 TABLES
task_dict = {} 
all_results = {} 
for algo in ["Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned",\
             "Try2Ablation_nothresh",\
             'GMN_embed','GMN_match','IsoNet_node','IsoNet_edge',"IsoNet_node_EdgeScore",\
             "SimGNN","GraphSim","GOTSim","NeuroMatch",\
             "AllLayersPos_IsoNet_node","IsoNet_baseline",\
             "AsymCrossSinkhorn_T3_ISONET_for_mcs",\
             "AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs",'GMN_embed_nomin','GMN_match_nomin']: 
    task_dict[algo] = {}
    all_results[algo] = {}
    for gt_mode in ['qap','gossip_qap']:
        task_dict[algo][gt_mode] = {}
        all_results[algo][gt_mode] = {}
        for training_mode in ['mse']:
            task_dict[algo][gt_mode][training_mode] = {}
            all_results[algo][gt_mode][training_mode] = {}

for dataset in ['ptc_mm_500qgrlarge','ptc_mr_500qgrlarge','ptc_fm_500qgrlarge',\
                            'ptc_fr_500qgrlarge','msrc_21_500qgrlarge',\
                            'dd_500qgrlarge', 'cox2_500qgrlarge']:
    for gt_mode in ['qap','gossip_qap']:#,'glasgow']:
        task_dict['GMN_embed'][gt_mode]['mse'][dataset] = "T3_GMN_embed_newrun1_FlagOff"
        task_dict['GMN_match'][gt_mode]['mse'][dataset] = "T3_GMN_match_newrun1_FlagOff"
        task_dict['GMN_embed_nomin'][gt_mode]['mse'][dataset] = "T5_GMN_embed_nomin_newrun1_FlagOff"
        task_dict['GMN_match_nomin'][gt_mode]['mse'][dataset] = "T5_GMN_match_nomin_newrun1_FlagOff"
        task_dict['SimGNN'][gt_mode]['mse'][dataset] = "SimGNN_for_mcs_newrun1_FlagOff"
        task_dict['GraphSim'][gt_mode]['mse'][dataset] = "GraphSim_for_mcs_newrun1_FlagOff"
        task_dict['GOTSim'][gt_mode]['mse'][dataset] = "GOTSim_for_mcs_newrun1_FlagOff"
        task_dict['NeuroMatch'][gt_mode]['mse'][dataset] = "NeuroMatch_for_mcs_newrun1_FlagOff"
        task_dict['IsoNet_node'][gt_mode]['mse'][dataset] = "T3_ISONET_for_mcs_newrun1_FlagOff_nodeEmb_td16_lr1e-3_noise0"
        task_dict['AllLayersPos_IsoNet_node'][gt_mode]['mse'][dataset] = "AllLayersPos_T3_ISONET_for_mcs_newrun1_FlagOff_nodeEmb_td16_lr1e-3_noise0"
        task_dict['AsymCrossSinkhorn_T3_ISONET_for_mcs'][gt_mode]['mse'][dataset] = "AsymCrossSinkhorn_T3_ISONET_for_mcs_newrun1_FlagOff_nodeEmb_td16_lr1e-3_noise0"
        task_dict['AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs'][gt_mode]['mse'][dataset] = "AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs_newrun1_FlagOff_nodeEmb_td16_lr1e-3_noise0"
        task_dict['IsoNet_edge'][gt_mode]['mse'][dataset] = "ISONET_for_mcs_newrun1_FlagOff_edgeEmb_td16_lr1e-3_noise0"
        task_dict['IsoNet_baseline'][gt_mode]['mse'][dataset] = "ISONET_baseline_newrun1_FlagOff_edgeEmb_td16_lr1e-3_noise0"
        task_dict['IsoNet_node_EdgeScore'][gt_mode]['mse'][dataset] = "IsoNetVar29ForMcs_newrun1_FlagOff_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
        task_dict['IsoNetGossip_node_EdgeScore_LC1_temp5e-1'][gt_mode]['mse'][dataset] = "IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp5e-1_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
        task_dict['Try2Ablation_nothresh'][gt_mode]['mse'][dataset] = "Try2Abaltion_NoThresh_IsoNetGossipVar29ForMcs_GossipVector_newrun1_FlagOff_temp1e+0_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"

task_dict['Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned'][gt_mode]['mse']['ptc_mm_500qgrlarge'] = \
"Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp7e-1_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
task_dict['Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned'][gt_mode]['mse']['ptc_mr_500qgrlarge'] = \
"Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp1e-1_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
task_dict['Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned'][gt_mode]['mse']['ptc_fm_500qgrlarge'] = \
"Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp8e-1_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
task_dict['Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned'][gt_mode]['mse']['ptc_fr_500qgrlarge'] = \
"Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp14e-1_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
task_dict['Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned'][gt_mode]['mse']['msrc_21_500qgrlarge'] = \
"Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp1e+0_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
task_dict['Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned'][gt_mode]['mse']['dd_500qgrlarge'] = \
"Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp1e+1_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
task_dict['Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned'][gt_mode]['mse']['cox2_500qgrlarge'] = \
"Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp11e-1_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"

     
for dataset in ['ptc_mm_500qgrlarge','ptc_mr_500qgrlarge','ptc_fm_500qgrlarge','ptc_fr_500qgrlarge']:
    for gt_mode in ['qap','gossip_qap']:#,'glasgow']:
        task_dict['GMN_match'][gt_mode]['mse'][dataset] = "T3_GMN_match_newrun1"
        task_dict['SimGNN'][gt_mode]['mse'][dataset] = "SimGNN_for_mcs_newrun1"
        task_dict['GraphSim'][gt_mode]['mse'][dataset] = "GraphSim_for_mcs_newrun1"
        task_dict['GOTSim'][gt_mode]['mse'][dataset] = "GOTSim_for_mcs_newrun1"
        task_dict['NeuroMatch'][gt_mode]['mse'][dataset] = "NeuroMatch_for_mcs_newrun1"
    

In [7]:
s = time.time()
for gmode in ['gossip_qap','qap']:
    for tmode in ["mse"]:
        av.gt_mode = gmode
        av.training_mode = tmode
        
        for algo in [\
            "SimGNN","GraphSim","NeuroMatch","GOTSim"]: 

            if algo == "IsoNetGossip_node_EdgeScore_LC1_temp5e-1":
                av.GOSSIP_TEMP = 0.5
            else:
                av.GOSSIP_TEMP=1.0
            if "IsoNetGossip" in algo and gmode == "qap":
                continue
            for dataset in ['ptc_mm_500qgrlarge','ptc_mr_500qgrlarge','ptc_fm_500qgrlarge',\
                            'ptc_fr_500qgrlarge','msrc_21_500qgrlarge',\
                            'dd_500qgrlarge', 'cox2_500qgrlarge']:
                print(algo)
                if algo == "Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned": 
                    if dataset == "ptc_mm_500qgrlarge": 
                        av.GOSSIP_TEMP = 0.7
                    elif dataset == "ptc_mr_500qgrlarge": 
                        av.GOSSIP_TEMP = 0.1
                    elif dataset == "ptc_fm_500qgrlarge": 
                        av.GOSSIP_TEMP = 0.8
                    elif dataset == "ptc_fr_500qgrlarge": 
                        av.GOSSIP_TEMP = 1.4
                    elif dataset == "msrc_21_500qgrlarge": 
                        av.GOSSIP_TEMP = 1
                    elif dataset == "dd_500qgrlarge": 
                        av.GOSSIP_TEMP = 10
                    elif dataset == "cox2_500qgrlarge": 
                        av.GOSSIP_TEMP = 1.1
                try:
                    all_results[algo][av.gt_mode][av.training_mode][dataset] =\
                        get_result(task_dict[algo][av.gt_mode][av.training_mode][dataset],dataset)
                except:
                    print("Try Again!")
                
print(time.time()-s)

SimGNN
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/

Try Again!
GraphSim
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Dataset

/mnt/infonas/data/velugoti/envs/genv/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


GraphSim
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/split

NeuroMatch
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/spl

Try Again!
GOTSim
loading train query graphs from ./Datasets/mcs/splits/train/train_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading train mcs info from ./Datasets/mcs/splits/train/train_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading train qap mcs info from ./Datasets/mcs/splits/train/train_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading train gossip qap mcs info from ./Datasets/mcs/splits/train/train_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from

loading best validated model from ./bestValidationModels/SimGNN_for_mcs_newrun1_gt_mode_qap_trMode_mse_ptc_mm_500qgrlarge
SimGNN
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./

GraphSim
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/split

NeuroMatch
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/spl

GOTSim
loading train query graphs from ./Datasets/mcs/splits/train/train_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading train mcs info from ./Datasets/mcs/splits/train/train_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading train qap mcs info from ./Datasets/mcs/splits/train/train_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading train gossip qap mcs info from ./Datasets/mcs/splits/train/train_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets

In [9]:
s = time.time()
for gmode in ['gossip_qap','qap']:
    for tmode in ["mse"]:
        av.gt_mode = gmode
        av.training_mode = tmode
        
        for algo in [\
            "SimGNN","GraphSim","NeuroMatch","GOTSim"]: 

            if algo == "IsoNetGossip_node_EdgeScore_LC1_temp5e-1":
                av.GOSSIP_TEMP = 0.5
            else:
                av.GOSSIP_TEMP=1.0
            if "IsoNetGossip" in algo and gmode == "qap":
                continue
            for dataset in ['ptc_mm_500qgrlarge','ptc_mr_500qgrlarge','ptc_fm_500qgrlarge',\
                            'ptc_fr_500qgrlarge','msrc_21_500qgrlarge',\
                            'dd_500qgrlarge', 'cox2_500qgrlarge']:
                print(algo)
                if algo == "Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned": 
                    if dataset == "ptc_mm_500qgrlarge": 
                        av.GOSSIP_TEMP = 0.7
                    elif dataset == "ptc_mr_500qgrlarge": 
                        av.GOSSIP_TEMP = 0.1
                    elif dataset == "ptc_fm_500qgrlarge": 
                        av.GOSSIP_TEMP = 0.8
                    elif dataset == "ptc_fr_500qgrlarge": 
                        av.GOSSIP_TEMP = 1.4
                    elif dataset == "msrc_21_500qgrlarge": 
                        av.GOSSIP_TEMP = 1
                    elif dataset == "dd_500qgrlarge": 
                        av.GOSSIP_TEMP = 10
                    elif dataset == "cox2_500qgrlarge": 
                        av.GOSSIP_TEMP = 1.1
                try:
                    all_results[algo][av.gt_mode][av.training_mode][dataset] =\
                        get_result(task_dict[algo][av.gt_mode][av.training_mode][dataset],dataset)
                except:
                    print("Try Again!")
                
print(time.time()-s)

SimGNN
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/

GraphSim
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/split

NeuroMatch
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/spl

GOTSim
loading train query graphs from ./Datasets/mcs/splits/train/train_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading train mcs info from ./Datasets/mcs/splits/train/train_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading train qap mcs info from ./Datasets/mcs/splits/train/train_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading train gossip qap mcs info from ./Datasets/mcs/splits/train/train_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets

Try Again!
SimGNN
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/

loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/GraphSim_for_mcs_newr

loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/NeuroMatch_for_mcs_newrun1_gt_mode_qap_trMode_mse_ptc_mm_500qgrlarge
NeuroMatch
loading test query graphs from ./Datasets/m

loading train gossip qap mcs info from ./Datasets/mcs/splits/train/train_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/GOTSim_for_mcs_newrun1_gt_mode_qap_trMode_mse_ptc_mm_500qgrlarge
Try Again!
GOTSim
loading train query graphs from ./Datasets/mcs/splits/train/train_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading train mcs inf

In [10]:
s = time.time()
for gmode in ['gossip_qap','qap']:
    for tmode in ["mse"]:
        av.gt_mode = gmode
        av.training_mode = tmode
        
        for algo in [\
            "IsoNetGossip_node_EdgeScore_LC1_temp5e-1",\
             "Try1_IsoNetGossip_node_EdgeScore_LC1_temp1e+0",\
             "Try2_IsoNetGossip_node_EdgeScore_LC1_temp1e+0",\
             'GMN_embed','GMN_match',\
             'GMN_embed_nomin','GMN_match_nomin',\
             'IsoNet_node','IsoNet_edge',"IsoNet_node_EdgeScore",\
             "Try2Ablation_nothresh","IsoNet_baseline",\
            # "SimGNN","GraphSim","NeuroMatch",\
            #"GOTSim",\
              "AllLayersPos_IsoNet_node",\
            "CrossSinkhorn_T3_ISONET_for_mcs",\
             "CrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs",\
            "AsymCrossSinkhorn_T3_ISONET_for_mcs",\
             "AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs"]: 

            if algo == "IsoNetGossip_node_EdgeScore_LC1_temp5e-1":
                av.GOSSIP_TEMP = 0.5
            else:
                av.GOSSIP_TEMP=1.0
            if "IsoNetGossip" in algo and gmode == "qap":
                continue
            if ("Try2Ablation_nothresh" in model) and (gt_mode == "qap"):
                continue
            for dataset in ['ptc_mm_500qgrlarge','ptc_mr_500qgrlarge','ptc_fm_500qgrlarge',\
                            'ptc_fr_500qgrlarge','msrc_21_500qgrlarge',\
                            'dd_500qgrlarge', 'cox2_500qgrlarge']:
                print(algo)
                if algo == "Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned": 
                    if dataset == "ptc_mm_500qgrlarge": 
                        av.GOSSIP_TEMP = 0.7
                    elif dataset == "ptc_mr_500qgrlarge": 
                        av.GOSSIP_TEMP = 0.1
                    elif dataset == "ptc_fm_500qgrlarge": 
                        av.GOSSIP_TEMP = 0.8
                    elif dataset == "ptc_fr_500qgrlarge": 
                        av.GOSSIP_TEMP = 1.4
                    elif dataset == "msrc_21_500qgrlarge": 
                        av.GOSSIP_TEMP = 1
                    elif dataset == "dd_500qgrlarge": 
                        av.GOSSIP_TEMP = 10
                    elif dataset == "cox2_500qgrlarge": 
                        av.GOSSIP_TEMP = 1.1
                all_results[algo][av.gt_mode][av.training_mode][dataset] =\
                    get_result(task_dict[algo][av.gt_mode][av.training_mode][dataset],dataset)
print(time.time()-s)

IsoNetGossip_node_EdgeScore_LC1_temp5e-1
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus 

loading corpus graphs from ./Datasets/mcs/splits/cox2_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_cox2_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_cox2_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_cox2_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_cox2_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/cox2_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp5e-1_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0_gt_mode_gossip_qap_trMode_mse_cox2_500qgrlarge
Try1_IsoNetGossip_node_EdgeScore_LC1_temp1e+0
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from 

loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_dd_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/dd_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/Try1_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_FlagOff_temp1e+0_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0_gt_mode_gossip_qap_trMode_mse_dd_500qgrlarge
Try1_IsoNetGossip_node_EdgeScore_LC1_temp1e+0
loading test query graphs from ./Datasets/mcs/splits/test/test_cox2_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_cox2_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_cox2_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_cox2_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/cox2_500qgrlarge80k_corpus_subgraphs.pkl
loading val query gr

Try2_IsoNetGossip_node_EdgeScore_LC1_temp1e+0
loading test query graphs from ./Datasets/mcs/splits/test/test_dd_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_dd_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_dd_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_dd_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/dd_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_dd_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_dd_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_dd_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_dd_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/spli

loading corpus graphs from ./Datasets/mcs/splits/msrc_21_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/T3_GMN_embed_newrun1_FlagOff_gt_mode_gossip_qap_trMode_mse_msrc_21_500qgrlarge
GMN_embed
loading test query graphs from ./Datasets/mcs/splits/test/test_dd_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_dd_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_dd_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_dd_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/dd_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_dd_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_dd_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val

...

loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/CrossSinkhorn_T3_ISONET_for_mcs_newrun1_FlagOff_nodeEmb_td16_lr1e-3_noise0_gt_mode_qap_trMode_mse_ptc_mr_500qgrlarge
CrossSinkhorn_T3_ISONET_for_mcs
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarg

CrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mr_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mr_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading cor

loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/AsymCrossSinkhorn_T3_ISONET_for_mcs_newrun1_FlagOff_nodeEmb_td16_lr1e-3_noise0_gt_mode_qap_trMode_mse_ptc_mm_500qgrlarge
AsymCrossSinkhorn_T3_ISONET_for_mcs
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mr_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/tes

loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs_newrun1_FlagOff_nodeEmb_td16_lr1e-3_noise0_gt_mode_qap_trMode_mse_ptc_mm_5

loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_cox2_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/cox2_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs_newrun1_FlagOff_nodeEmb_td16_lr1e-3_noise0_gt_mode_qap_trMode_mse_cox2_500qgrlarge
17305.735476255417


In [2]:
#Taking care of per query stuff + standard error
s = time.time()
for k1 in all_results.keys():
    for k2 in all_results[k1].keys():
        for k3 in all_results[k1][k2].keys():
            if all_results[k1][k2][k3] == {}:
                continue
            for k4 in  all_results[k1][k2][k3].keys():
                v1 = []
                for k5 in range(len(all_results[k1][k2][k3][k4])):
                    all_info = all_results[k1][k2][k3][k4][k5]
                    all_p = all_info[5]
                    all_t = all_info[6]
                    assert(len(all_p)==len(all_t))
                    assert(len(all_p)==80000)
                    qchunk_all_p = torch.chunk(all_p,100)
                    qchunk_all_t = torch.chunk(all_t,100)
                    
                    all_ktau = [] 
                    all_pr = []
                    all_mse = []
                    for idx in range(100):
                        all_mse.append(torch.nn.functional.mse_loss(qchunk_all_t[idx].cpu(),\
                                                qchunk_all_p[idx].cpu(),reduction="mean").item() )
                        all_ktau.append(kendalltau(qchunk_all_p[idx].cpu().tolist(),\
                        qchunk_all_t[idx].cpu().tolist())[0])
                        all_pr.append(all_pair_similarity_reward1(qchunk_all_p[idx],\
                                qchunk_all_t[idx],set(qchunk_all_t[idx].tolist())).item())
                    assert np.isclose(np.nanmean(all_mse),all_info[1])
                    #assert (np.nanmean(all_ktau) == all_info[7])
                    #assert (np.nanmean(all_pr) == all_info[8])
                    v1.append(all_info + (np.nanmean(all_ktau),np.nanmean(all_pr))+\
                                          ((np.nanstd(all_mse)/np.sqrt(np.count_nonzero(~np.isnan(all_mse)))),\
                                          (np.nanstd(all_ktau)/np.sqrt(np.count_nonzero(~np.isnan(all_ktau)))),\
                                          (np.nanstd(all_pr))/np.sqrt(np.count_nonzero(~np.isnan(all_pr)))))
                all_results[k1][k2][k3][k4] = v1  
                
print(time.time()-s)   
#Commented after first run
# pickle.dump(all_results, open("./Datasets/FlagOff_all_results_returnAll_All7_withperQktau_stdErr.pkl","wb"))

In [129]:
# all_results = pickle.load(open("./Datasets/FlagOff_all_results_returnAll_All7.pkl","rb"))
all_results = pickle.load(open("./Datasets/FlagOff_all_results_returnAll_All7_withperQktau_stdErr.pkl","rb"))


In [197]:
def get_first_and_second(results_dict, mode, gt_mode, tr_mode, score, early_list, late_list):
    m = 1 if mode=="test" else 0
    if score == "mse":
        offset1 = 1
        initV = 10000
    elif score == "ktau":
        offset1 = 7
        initV = -10000
    elif score == "pair":
        offset1 = 8
        initV = -10000

    max_data = {}
    max2_data = {}

    for gt_mode in ["qap","gossip_qap"]:
        max_data[gt_mode] = {}
        for dataset in ['msrc_21_500qgrlarge','ptc_mm_500qgrlarge','ptc_fr_500qgrlarge',\
                        'ptc_mr_500qgrlarge','ptc_fm_500qgrlarge','cox2_500qgrlarge','dd_500qgrlarge']:
            max_data[gt_mode][dataset] =  {} 
            max_data[gt_mode][dataset]["late"] = initV
            for model in late_list: 
                if ("IsoNetGossip" in model) and (gt_mode == "qap"):
                    continue
                if ("AllLayersPos_IsoNet_node" in model) and (gt_mode == "gossip_qap"):
                    continue    
                if ("Try2Ablation_nothresh" in model) and (gt_mode == "qap"):
                    continue
                if score == "mse" :
                    if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]<max_data[gt_mode][dataset]["late"]: 
                        max_data[gt_mode][dataset]["late"] = results_dict[model][gt_mode][tr_mode][dataset][m][offset1]
                elif score == "ktau" or score == "pair":
                    if  results_dict[model][gt_mode][tr_mode][dataset][m][offset1]>max_data[gt_mode][dataset]["late"]: 
                        max_data[gt_mode][dataset]["late"] = results_dict[model][gt_mode][tr_mode][dataset][m][offset1]
            max_data[gt_mode][dataset]["early"] = initV
            for model in early_list: 
                if ("IsoNetGossip" in model) and (gt_mode == "qap"):
                    continue
                if ("AllLayersPos_IsoNet_node" in model) and (gt_mode == "gossip_qap"):
                    continue   
                if ("IsoNet_node" == model) and (gt_mode == "gossip_qap"):
                    continue   
                if ("Try2Ablation_nothresh" in model) and (gt_mode == "qap"):
                    continue
                if ("IsoNet_node_EdgeScore" in model) and (gt_mode == "qap"):
                    continue
                if score == "mse" :
                    if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]<max_data[gt_mode][dataset]["early"]: 
                        max_data[gt_mode][dataset]["early"] = results_dict[model][gt_mode][tr_mode][dataset][m][offset1]
                elif score == "ktau" or score == "pair":
                    if  results_dict[model][gt_mode][tr_mode][dataset][m][offset1]>max_data[gt_mode][dataset]["early"]: 
                        max_data[gt_mode][dataset]["early"] = results_dict[model][gt_mode][tr_mode][dataset][m][offset1]

                        
    for gt_mode in ["qap","gossip_qap"]:
        max2_data[gt_mode] = {}
        for dataset in ['msrc_21_500qgrlarge','ptc_mm_500qgrlarge','ptc_fr_500qgrlarge',\
                        'ptc_mr_500qgrlarge','ptc_fm_500qgrlarge','cox2_500qgrlarge','dd_500qgrlarge']:
            max2_data[gt_mode][dataset] =  {} 
            max2_data[gt_mode][dataset]["late"] = initV
            for model in late_list: 
                if ("IsoNetGossip" in model) and (gt_mode == "qap"):
                    continue
                if ("AllLayersPos_IsoNet_node" in model) and (gt_mode == "gossip_qap"):
                    continue   
                if ("IsoNet_node" == model) and (gt_mode == "gossip_qap"):
                    continue   
                if ("Try2Ablation_nothresh" in model) and (gt_mode == "qap"):
                    continue
                if ("IsoNet_node_EdgeScore" in model) and (gt_mode == "qap"):
                    continue
                if score == "mse" :
                    if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]<max2_data[gt_mode][dataset]["late"]:
                        if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]!= max_data[gt_mode][dataset]["late"]:
                            max2_data[gt_mode][dataset]["late"] = results_dict[model][gt_mode][tr_mode][dataset][m][offset1]
                elif score == "ktau" or score == "pair":
                    if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]>max2_data[gt_mode][dataset]["late"]:
                        if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]!= max_data[gt_mode][dataset]["late"]:
                            max2_data[gt_mode][dataset]["late"] = results_dict[model][gt_mode][tr_mode][dataset][m][offset1]
            max2_data[gt_mode][dataset]["early"] = initV
            for model in early_list: 
                if ("IsoNetGossip" in model) and (gt_mode == "qap"):
                    continue
                if ("AllLayersPos_IsoNet_node" in model) and (gt_mode == "gossip_qap"):
                    continue    
                if ("Try2Ablation_nothresh" in model) and (gt_mode == "qap"):
                    continue
                if score == "mse" :
                    if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]<max2_data[gt_mode][dataset]["early"]:
                        if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]!= max_data[gt_mode][dataset]["early"]:
                            max2_data[gt_mode][dataset]["early"] = results_dict[model][gt_mode][tr_mode][dataset][m][offset1]
                elif score == "ktau" or score == "pair":
                    if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]>max2_data[gt_mode][dataset]["early"]:
                        if results_dict[model][gt_mode][tr_mode][dataset][m][offset1]!= max_data[gt_mode][dataset]["early"]:
                            max2_data[gt_mode][dataset]["early"] = results_dict[model][gt_mode][tr_mode][dataset][m][offset1]

                        
    return max_data,max2_data    


In [198]:
latex_names = {}
latex_names["SimGNN"] = "&SimGNN"
latex_names["GraphSim"] = "&GraphSim"
latex_names["GOTSim"] = "&GOTSim"
latex_names["NeuroMatch"] = "&NeuroMatch"
latex_names["IsoNet_baseline"] = "&IsoNet"
latex_names["IsoNet_edge"] = "& IsoNet (MCS)"
latex_names["IsoNet_node"] = "&\\mcesabbrv\\  (final layer)"
latex_names["IsoNet_node_EdgeScore"] = "&\\mccsabbrv\\ (no gossip) "
latex_names["Try2Ablation_nothresh"] = "&\\mccsabbrv\\ (no \\textsc{Noise Filter}) "
latex_names["GMN_embed_nomin"] = "& \\gmne"
latex_names["GMN_embed"] = "& \\gmnemin"
latex_names["Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned"] = "&\\mccs"
latex_names["AllLayersPos_IsoNet_node"] = "&\\mces"
latex_names["GMN_match_nomin"] = "& \\gmnm"
latex_names["GMN_match"] = "& \\gmnmmin"
latex_names["AsymCrossSinkhorn_T3_ISONET_for_mcs"] = "&\\early "
latex_names["AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs"] = "& \\earlyabbrv\\ (all layers) "




def print_part_app_latex_table(results_dict, mode, gt_mode, tr_mode, score,early_list,late_list):
    max_data,max2_data = get_first_and_second(results_dict, mode, gt_mode, tr_mode, score,early_list,late_list)
    m = 1 if mode=="test" else 0
    offset1 = -1
    offset2 = -1
    if score == "mse":
        print("& \multirow{2}{*}{\\textbf{MCES}}& \multicolumn{7}{c}{\\textbf{MSE  (lower is better) } }  \\\\")
        offset1 = 1
        offset2 = 9
    elif score == "ktau":
        print("& \multirow{2}{*}{\\textbf{MCES}}& \multicolumn{7}{c}{\\textbf{KTau  (higher is better) } }  \\\\")
        offset1 = 7
        offset2 = 10
    elif score == "pair":
        print("& \multirow{2}{*}{\\textbf{MCES}}& \multicolumn{7}{c}{\\textbf{PairRank  (higher is better) } }  \\\\")
        offset1 = 8
        offset2 = 11
    lene = len(early_list)
    lenl = len(late_list)
    print("& & \msrc & \mm & \\fr & \mr  & \\fm & \cox & \dd \\\\ \hline\hline")
    print("%%%    ")
    print("\\multirow{" +str(lenl) + "}{.3cm}{\\begin{sideways}\\textbf{Late}\end{sideways}}\ldelim\{{" +str(lenl) + "}{.1mm}")
    
    for s in late_list: 
        if ("IsoNetGossip" in s) and (gt_mode == "qap"):
            continue
        if ("AllLayersPos_IsoNet_node" in s) and (gt_mode == "gossip_qap"):
            continue   
        if ("IsoNet_node" == s) and (gt_mode == "gossip_qap"):
            continue   
        if ("Try2Ablation_nothresh" in s) and (gt_mode == "qap"):
            continue
        if ("IsoNet_node_EdgeScore" in s) and (gt_mode == "qap"):
            continue
        res = results_dict[s][gt_mode][tr_mode]
        s1 = latex_names[s]
        for dataset in ['msrc_21_500qgrlarge','ptc_mm_500qgrlarge','ptc_fr_500qgrlarge',\
                        'ptc_mr_500qgrlarge','ptc_fm_500qgrlarge','cox2_500qgrlarge','dd_500qgrlarge']:
            if res[dataset][m][offset1] == max_data[gt_mode][dataset]['late']:
                s1 = s1 + " & \\lfirst{" + "{:.3f}".format(res[dataset][m][offset1]) + "$\pm$" + "{:.3f}".format(res[dataset][m][offset2]) + "}"
            elif res[dataset][m][offset1] == max2_data[gt_mode][dataset]['late']:
                s1 = s1 + " & \\second{" + "{:.3f}".format(res[dataset][m][offset1]) + "$\pm$" + "{:.3f}".format(res[dataset][m][offset2]) + "}"
            else:
                s1 = s1 + " & " + "{:.3f}".format(res[dataset][m][offset1]) + "$\pm$" + "{:.3f}".format(res[dataset][m][offset2])
            # 0:ndcg 1:mse 2:rankcorr 3:mae 4:pairReward 5:all_pred 6: all_targe 7:avgktau  8:avg pr 
            # 9:stderr mse 10:stderr ktau 11:stderr pr
        print(s1,"\\\\")
    print("\\hline")
                                                                         
    print("\\multirow{" +str(lene) + "}{.3cm}{\\begin{sideways}\\textbf{Early}\end{sideways}}\ldelim\{{" +str(lene) + "}{.1mm}")
     
    for s in early_list: 
        res = results_dict[s][gt_mode][tr_mode]
        s1 = latex_names[s]
        for dataset in ['msrc_21_500qgrlarge','ptc_mm_500qgrlarge','ptc_fr_500qgrlarge',\
                        'ptc_mr_500qgrlarge','ptc_fm_500qgrlarge','cox2_500qgrlarge','dd_500qgrlarge']:
            if res[dataset][m][offset1] == max_data[gt_mode][dataset]['early']:
                s1 = s1 + " & \\first{" + "{:.3f}".format(res[dataset][m][offset1]) + "$\pm$" + "{:.3f}".format(res[dataset][m][offset2]) + "}"
            elif res[dataset][m][offset1] == max2_data[gt_mode][dataset]['early']:
                s1 = s1 + " & \\esecond{" + "{:.3f}".format(res[dataset][m][offset1]) + "$\pm$" + "{:.3f}".format(res[dataset][m][offset2]) + "}"
            else:
                s1 = s1 + " & " + "{:.3f}".format(res[dataset][m][offset1]) + "$\pm$" + "{:.3f}".format(res[dataset][m][offset2])
            # 0:ndcg 1:mse 2:rankcorr 3:mae 4:pairReward 5:all_pred 6: all_targe 7:avgktau  8:avg pr 
            # 9:stderr mse 10:stderr ktau 11:stderr pr
        print(s1,"\\\\")
    print("[0.4ex]\hline  \\\\[-1ex]")                                                                        
                                                                         

        
def print_app_latex_table(results_dict, mode, gt_mode, tr_mode, score,early_list,late_list):
    
    print("\\begin{table*}[t]")
    print("\\begin{center}")
    print("\\maxsizebox{0.98\hsize}{!}{  \\tabcolsep 4pt ")
    print("\\begin{tabular}{p{.4cm}l|ccccccc}")
    print("\\hline")    
    
    print_part_app_latex_table(results_dict, mode, "qap", tr_mode, score,early_list,late_list)
    print("\\hline")             
    print_part_app_latex_table(results_dict, mode, "gossip_qap", tr_mode, score,early_list,late_list)
        
    print("\\end{tabular} } ")
    print("\\end{center} ")
    print("\\caption{All 7 dataset results - TODO} ")
    print("\\label{tab:TODO} ")
    print("\\end{table*} ")            


<font size="6"> Data for central table \
    Table 1 (Main) Tables 7,8,9 (Appendix)

In [179]:
late_list = ["SimGNN","GraphSim","GOTSim","NeuroMatch","IsoNet_baseline",'GMN_embed_nomin',\
              "Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned",\
              "AllLayersPos_IsoNet_node"]
early_list = ['GMN_match_nomin',"AsymCrossSinkhorn_T3_ISONET_for_mcs"]

print_app_latex_table(all_results, "test","qap","mse","mse",early_list,late_list)

\begin{table*}[t]
\begin{center}
\maxsizebox{0.98\hsize}{!}{  \tabcolsep 4pt 
\begin{tabular}{p{.4cm}l|ccccccc}
\hline
& \multirow{2}{*}{\textbf{MCES}}& \multicolumn{7}{c}{\textbf{MSE  (lower is better) } }  \\
& & \msrc & \mm & \fr & \mr  & \fm & \cox & \dd \\ \hline\hline
%%%    
\multirow{8}{.3cm}{\begin{sideways}\textbf{Late}\end{sideways}}\ldelim\{{8}{.1mm}
&SimGNN & 0.910$\pm$0.044 & 0.302$\pm$0.009 & 0.355$\pm$0.021 & 0.337$\pm$0.015 & 0.331$\pm$0.014 & 0.281$\pm$0.012 & 1.210$\pm$0.073 \\
&GraphSim & 0.629$\pm$0.028 & 0.274$\pm$0.010 & 0.282$\pm$0.012 & 0.274$\pm$0.010 & 0.261$\pm$0.009 & 0.249$\pm$0.009 & 0.881$\pm$0.081 \\
&GOTSim & 0.496$\pm$0.020 & 0.343$\pm$0.046 & 0.326$\pm$0.020 & 0.320$\pm$0.031 & 0.359$\pm$0.047 & 0.328$\pm$0.015 & 0.628$\pm$0.037 \\
&NeuroMatch & 0.582$\pm$0.082 & 0.308$\pm$0.059 & 0.282$\pm$0.055 & 0.795$\pm$0.606 & 0.604$\pm$0.322 & 0.269$\pm$0.072 & 2.827$\pm$2.281 \\
&SimGNN & \second{0.276$\pm$0.007} & \second{0.225$\pm$0.009} & \second{0.220$\pm

<font size="6"> Data for effect of MCS layer \
    Table 2 (Main) Tables 10,11 (Appendix)

In [185]:
late_list = ['GMN_embed_nomin','GMN_embed',"IsoNet_baseline","IsoNet_edge",\
              "Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned",\
              "AllLayersPos_IsoNet_node"]
early_list = ['GMN_match_nomin','GMN_match',"AsymCrossSinkhorn_T3_ISONET_for_mcs"]

print_app_latex_table(all_results, "test","qap","mse","ktau",early_list,late_list)

\begin{table*}[t]
\begin{center}
\maxsizebox{0.98\hsize}{!}{  \tabcolsep 4pt 
\begin{tabular}{p{.4cm}l|ccccccc}
\hline
& \multirow{2}{*}{\textbf{MCES}}& \multicolumn{7}{c}{\textbf{KTau  (higher is better) } }  \\
& & \msrc & \mm & \fr & \mr  & \fm & \cox & \dd \\ \hline\hline
%%%    
\multirow{6}{.3cm}{\begin{sideways}\textbf{Late}\end{sideways}}\ldelim\{{6}{.1mm}
& \gmne & 0.627$\pm$0.005 & 0.416$\pm$0.013 & 0.468$\pm$0.011 & 0.456$\pm$0.012 & 0.456$\pm$0.015 & 0.466$\pm$0.014 & 0.635$\pm$0.010 \\
& \gmnemin & 0.666$\pm$0.004 & 0.556$\pm$0.006 & 0.565$\pm$0.007 & 0.584$\pm$0.006 & \second{0.590$\pm$0.005} & 0.555$\pm$0.010 & 0.690$\pm$0.004 \\
&IsoNet & 0.669$\pm$0.004 & 0.506$\pm$0.010 & 0.504$\pm$0.008 & 0.537$\pm$0.007 & 0.532$\pm$0.007 & 0.522$\pm$0.012 & 0.698$\pm$0.005 \\
& IsoNet (MCS) & \second{0.677$\pm$0.004} & \second{0.569$\pm$0.006} & \second{0.570$\pm$0.007} & \second{0.585$\pm$0.005} & 0.589$\pm$0.005 & \second{0.565$\pm$0.009} & \second{0.708$\pm$0.004} \\
&\mces & \lf

<font size="6"> Data for Ablation Study \
    Table 3 (Main) Tables 12,13 (Appendix)

In [200]:
late_list = ["IsoNet_node","AllLayersPos_IsoNet_node",\
               "IsoNet_node_EdgeScore","Try2Ablation_nothresh",\
            "Try2_IsoNetGossip_node_EdgeScore_LC1_tempTuned" ]
early_list = ["AsymCrossSinkhorn_AllLayersPos_T3_ISONET_for_mcs",\
"AsymCrossSinkhorn_T3_ISONET_for_mcs"]

print_app_latex_table(all_results, "test","qap","mse","ktau",early_list,late_list)

\begin{table*}[t]
\begin{center}
\maxsizebox{0.98\hsize}{!}{  \tabcolsep 4pt 
\begin{tabular}{p{.4cm}l|ccccccc}
\hline
& \multirow{2}{*}{\textbf{MCES}}& \multicolumn{7}{c}{\textbf{KTau  (higher is better) } }  \\
& & \msrc & \mm & \fr & \mr  & \fm & \cox & \dd \\ \hline\hline
%%%    
\multirow{5}{.3cm}{\begin{sideways}\textbf{Late}\end{sideways}}\ldelim\{{5}{.1mm}
&\mcesabbrv\  (final layer) & \lfirst{0.692$\pm$0.004} & \second{0.562$\pm$0.006} & \second{0.581$\pm$0.006} & \second{0.594$\pm$0.005} & \second{0.599$\pm$0.005} & \second{0.566$\pm$0.009} & \second{0.720$\pm$0.004} \\
&\mces & \second{0.691$\pm$0.004} & \lfirst{0.577$\pm$0.006} & \lfirst{0.588$\pm$0.006} & \lfirst{0.598$\pm$0.005} & \lfirst{0.610$\pm$0.004} & \lfirst{0.574$\pm$0.009} & \lfirst{0.724$\pm$0.004} \\
\hline
\multirow{2}{.3cm}{\begin{sideways}\textbf{Early}\end{sideways}}\ldelim\{{2}{.1mm}
& \earlyabbrv\ (all layers)  & \first{0.700$\pm$0.004} & \first{0.587$\pm$0.006} & \esecond{0.580$\pm$0.006} & \esecond{0.61

<font size="6"> Data for linear combinations of MCES and MCCS scores \
    Table 14 (Appendix)

In [8]:
av.gt_mode = "combo"
tdict_combo = {}
tdict_combo[0.3] = {}
tdict_combo[0.7] = {}

tdict_combo[0.3]['AllLayersPos_IsonetNode'] = "AllLayersPos_T3_ISONET_for_mcs_newrun1_combo3e-1_FlagOff_nodeEmb_td16_lr1e-3_noise0"
tdict_combo[0.7]['AllLayersPos_IsonetNode'] = "AllLayersPos_T3_ISONET_for_mcs_newrun1_combo7e-1_FlagOff_nodeEmb_td16_lr1e-3_noise0"

tdict_combo[0.3]['GMN_match'] = "T3_GMN_match_newrun1_combo3e-1_FlagOff"
tdict_combo[0.7]['GMN_match'] = "T3_GMN_match_newrun1_combo7e-1_FlagOff"
tdict_combo[0.3]['GMN_embed'] = "T3_GMN_embed_newrun1_combo3e-1_FlagOff"
tdict_combo[0.7]['GMN_embed'] = "T3_GMN_embed_newrun1_combo7e-1_FlagOff"

tdict_combo[0.3]['AsymCrosSink']  = "AsymCrossSinkhorn_T3_ISONET_for_mcs_newrun1_combo3e-1_FlagOff_nodeEmb_td16_lr1e-3_noise0"
tdict_combo[0.7]['AsymCrosSink']  = "AsymCrossSinkhorn_T3_ISONET_for_mcs_newrun1_combo7e-1_FlagOff_nodeEmb_td16_lr1e-3_noise0"

tdict_combo[0.3]['Try2_Gossip'] = "Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_combo3e-1_FlagOff_temp1e+0_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
tdict_combo[0.7]['Try2_Gossip'] = "Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_combo7e-1_FlagOff_temp1e+0_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"

tdict_combo[0.3]['Combo_late'] = "Combo_late_models_newrun1_combo3e-1_FlagOff_temp1e+0_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"
tdict_combo[0.7]['Combo_late'] = "Combo_late_models_newrun1_combo7e-1_FlagOff_temp1e+0_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0"


In [12]:
s = time.time()
result_combo = {}
av.gt_mode = "combo"
for frac in [0.3,0.7]:
    result_combo[frac] = {}
    av.COMBO = frac
    for algo in ['Try2_Gossip','AllLayersPos_IsonetNode','GMN_match','GMN_embed','AsymCrosSink','Combo_late']:
        result_combo[frac][algo] = {}
        for dataset in ['ptc_fm_500qgrlarge','ptc_mm_500qgrlarge','msrc_21_500qgrlarge']: 
            #if algo == "Try2_Gossip" and dataset == "ptc_fm_500qgrlarge":
            #   result_combo[frac][algo][dataset] = ""
            #else:
            result_combo[frac][algo][dataset] = get_result(tdict_combo[frac][algo],\
                                                   dataset)        
print(time.time()-s)

loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_fm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_fm_

loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_fm_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/T3_GMN_match_newrun1_combo3e-1_FlagOff_gt_mode_combo_trMode_mse_ptc_fm_500qgrlarge
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Da

loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/AsymCrossSinkhorn_T3_ISONET_for_mcs_newrun1_combo3e-1_FlagOff_nodeEmb_td16_lr1e-3_noise0_gt_mode_combo_trMode_mse_ptc_mm_500qgrlarge
loading test query graphs from ./Datasets/mcs/splits/test/test_msrc_21_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_msrc_21_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_msrc_21_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_msrc_21_500qgrlarge

loading test qap mcs info from ./Datasets/mcs/splits/test/test_msrc_21_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_msrc_21_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/msrc_21_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_msrc_21_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_msrc_21_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_msrc_21_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_msrc_21_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/msrc_21_500qgrlarge80k_corpus_subgraphs.pkl
loading best validated model from ./bestValidationModels/Try2_IsoNetGossipVar29ForMcs_GossipVectorLRLThreshold_newrun1_combo7e-1_FlagOff_temp1e+0_nodeEmbEdgeScoreAdjMask_td

loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_fm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_fm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_rel_qap_mcs.pkl
loading val gossip qap mcs info from ./Datasets/mcs/splits/val/val_ptc_fm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_fm_

loading best validated model from ./bestValidationModels/Combo_late_models_newrun1_combo7e-1_FlagOff_temp1e+0_nodeEmbEdgeScoreAdjMask_td16_lr1e-3_noise0_gt_mode_combo_trMode_mse_ptc_fm_500qgrlarge
loading test query graphs from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading test mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading test qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_qap_mcs.pkl
loading test gossip qap mcs info from ./Datasets/mcs/splits/test/test_ptc_mm_500qgrlarge80k_rel_gossip_qap_mcs.pkl
loading corpus graphs from ./Datasets/mcs/splits/ptc_mm_500qgrlarge80k_corpus_subgraphs.pkl
loading val query graphs from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_enhanced_query_subgraphs.pkl
loading val mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_500qgrlarge80k_rel_mccreesh_mcs.pkl
loading val qap mcs info from ./Datasets/mcs/splits/val/val_ptc_mm_50

In [23]:
mode = "test"
m = 1 if mode=="test" else 0

for dataset in ['ptc_mm_500qgrlarge','msrc_21_500qgrlarge']: 
  for frac in [0.3,0.7]:
    for algo in ['GMN_embed','AllLayersPos_IsonetNode','Try2_Gossip', 'Combo_late', 'GMN_match','AsymCrosSink']:
        print(dataset, frac, algo ,result_combo[frac][algo][dataset][m][1] )


ptc_mm_500qgrlarge 0.3 GMN_embed 0.14581601321697235
ptc_mm_500qgrlarge 0.3 AllLayersPos_IsonetNode 0.14254571497440338
ptc_mm_500qgrlarge 0.3 Try2_Gossip 0.1724129170179367
ptc_mm_500qgrlarge 0.3 Combo_late 0.1447235643863678
ptc_mm_500qgrlarge 0.3 GMN_match 0.12934890389442444
ptc_mm_500qgrlarge 0.3 AsymCrosSink 0.12424161285161972
ptc_mm_500qgrlarge 0.7 GMN_embed 0.16544699668884277
ptc_mm_500qgrlarge 0.7 AllLayersPos_IsonetNode 0.17722120881080627
ptc_mm_500qgrlarge 0.7 Try2_Gossip 0.16649818420410156
ptc_mm_500qgrlarge 0.7 Combo_late 0.1400139182806015
ptc_mm_500qgrlarge 0.7 GMN_match 0.13022185862064362
ptc_mm_500qgrlarge 0.7 AsymCrosSink 0.12936805188655853
msrc_21_500qgrlarge 0.3 GMN_embed 0.15030431747436523
msrc_21_500qgrlarge 0.3 AllLayersPos_IsonetNode 0.12534311413764954
msrc_21_500qgrlarge 0.3 Try2_Gossip 1.0443044900894165
msrc_21_500qgrlarge 0.3 Combo_late 0.13017278909683228
msrc_21_500qgrlarge 0.3 GMN_match 0.12418679147958755
msrc_21_500qgrlarge 0.3 AsymCrosSink 0.12

In [27]:
all_results = result_combo

s = time.time()
for k1 in all_results.keys():
    for k2 in all_results[k1].keys():
        for k3 in all_results[k1][k2].keys():            
                v1 = []
                for k4 in range(len(all_results[k1][k2][k3])):
                    all_info = all_results[k1][k2][k3][k4]
                    all_p = all_info[5]
                    all_t = all_info[6]
                    assert(len(all_p)==len(all_t))
                    assert(len(all_p)==80000)
                    qchunk_all_p = torch.chunk(all_p,100)
                    qchunk_all_t = torch.chunk(all_t,100)
                    
                    all_ktau = [] 
                    all_pr = []
                    all_mse = []
                    for idx in range(100):
                        all_mse.append(torch.nn.functional.mse_loss(qchunk_all_t[idx].cpu(),\
                                                qchunk_all_p[idx].cpu(),reduction="mean").item() )
                        all_ktau.append(kendalltau(qchunk_all_p[idx].cpu().tolist(),\
                        qchunk_all_t[idx].cpu().tolist())[0])
                        all_pr.append(all_pair_similarity_reward1(qchunk_all_p[idx],\
                                qchunk_all_t[idx],set(qchunk_all_t[idx].tolist())).item())
                    assert np.isclose(np.nanmean(all_mse),all_info[1])
                    #assert (np.nanmean(all_ktau) == all_info[7])
                    #assert (np.nanmean(all_pr) == all_info[8])
                    v1.append(all_info + (np.nanmean(all_ktau),np.nanmean(all_pr))+\
                                          ((np.nanstd(all_mse)/np.sqrt(np.count_nonzero(~np.isnan(all_mse)))),\
                                          (np.nanstd(all_ktau)/np.sqrt(np.count_nonzero(~np.isnan(all_ktau)))),\
                                          (np.nanstd(all_pr))/np.sqrt(np.count_nonzero(~np.isnan(all_pr)))))
                all_results[k1][k2][k3] = v1  
                
print(time.time()-s)   
#commented after first run
#pickle.dump(all_results, open("./Datasets/comboResults_withperQktau_stdErr.pkl","wb"))

32.9273579120636


In [37]:
latex_names = {}
latex_names["GMN_embed"] = "& \\gmnemin"
latex_names["Try2_Gossip"] = "&\\mccs"
latex_names["AllLayersPos_IsonetNode"] = "&\\mces"
latex_names["GMN_match"] = "& \\gmnmmin"
latex_names["AsymCrosSink"] = "&\\early "
latex_names["Combo_late"] = "&COMBO "

def print_combo_latex_table(results_dict, mode):
    
    print("\\begin{table*}[t]")
    print("\\begin{center}")
    print("\\maxsizebox{0.98\hsize}{!}{  \\tabcolsep 4pt ")
    print("\\begin{tabular}{p{.4cm}l|cc|cc|cc|cc}")
    print("\\hline")    

    print("& \multirow{2}{*}{\\textbf{Combined}}& \multicolumn{4}{c|}{\\textbf{MSE  (lower is better) } } & \multicolumn{4}{c}{\\textbf{KTau (higher is better)}} \\\\\cline{3-10}")

    print("& & \multicolumn{2}{c|}{\msrc} & \multicolumn{2}{c|}{\mm}")
    print("& \multicolumn{2}{c|}{\msrc} & \multicolumn{2}{c}{\mm} \\\\\hline")
    print("& & 0.3 & 0.7 & 0.3 & 0.7") 
    print("& 0.3 & 0.7 & 0.3 & 0.7  \\\\ \hline\hline")
    m = 1 if mode=="test" else 0

    print("\\multirow{4}{.3cm}{\\begin{sideways}\\textbf{Late}\end{sideways}}\ldelim\{{4}{.1mm}")
    
    for s in ['GMN_embed','AllLayersPos_IsonetNode','Try2_Gossip', 'Combo_late']: 
        s1 = latex_names[s]
        for dataset in ['msrc_21_500qgrlarge','ptc_mm_500qgrlarge']:
          for frac in [0.3,0.7]:
            s1 = s1 + " & " + "{:.3f}".format( result_combo[frac][s][dataset][m][1]) +\
                    "$\pm$" + "{:.3f}".format( result_combo[frac][s][dataset][m][9])
        for dataset in ['msrc_21_500qgrlarge','ptc_mm_500qgrlarge']:
          for frac in [0.3,0.7]:
            s1 = s1 + " & " + "{:.3f}".format( result_combo[frac][s][dataset][m][7]) +\
                    "$\pm$" + "{:.3f}".format( result_combo[frac][s][dataset][m][10])
            # 0:ndcg 1:mse 2:rankcorr 3:mae 4:pairReward 5:all_pred 6: all_targe 7:avgktau  8:avg pr 
            # 9:stderr mse 10:stderr ktau 11:stderr pr
        print(s1,"\\\\")
    print("\\hline")
                                                                         
    print("\\multirow{2}{.3cm}{\\begin{sideways}\\textbf{Early}\end{sideways}}\ldelim\{{2}{.1mm}")
     
    for s in ['GMN_match','AsymCrosSink']: 
        s1 = latex_names[s]
        for dataset in ['msrc_21_500qgrlarge','ptc_mm_500qgrlarge']:
          for frac in [0.3,0.7]:
            s1 = s1 + " & " + "{:.3f}".format( result_combo[frac][s][dataset][m][1]) +\
                    "$\pm$" + "{:.3f}".format( result_combo[frac][s][dataset][m][9])
        for dataset in ['msrc_21_500qgrlarge','ptc_mm_500qgrlarge']:
          for frac in [0.3,0.7]:
            s1 = s1 + " & " + "{:.3f}".format( result_combo[frac][s][dataset][m][7]) +\
                    "$\pm$" + "{:.3f}".format( result_combo[frac][s][dataset][m][10])
            # 0:ndcg 1:mse 2:rankcorr 3:mae 4:pairReward 5:all_pred 6: all_targe 7:avgktau  8:avg pr 
            # 9:stderr mse 10:stderr ktau 11:stderr pr
        print(s1,"\\\\")
    print("[0.4ex]\hline  \\\\[-1ex]")                                                                        
    
        
    print("\\end{tabular} } ")
    print("\\end{center} ")
    print("\\caption{TODO} ")
    print("\\label{tab:TODO} ")
    print("\\end{table*} ")            


In [38]:
print_combo_latex_table(all_results, "test")

\begin{table*}[t]
\begin{center}
\maxsizebox{0.98\hsize}{!}{  \tabcolsep 4pt 
\begin{tabular}{p{.4cm}l|cc|cc|cc|cc}
\hline
& \multirow{2}{*}{\textbf{Combined}}& \multicolumn{4}{c|}{\textbf{MSE  (lower is better) } } & \multicolumn{4}{c}{\textbf{KTau (higher is better)}} \\\cline{3-10}
& & \multicolumn{2}{c}{\msrc} & \multicolumn{2}{c}{\mm}
& \multicolumn{2}{c}{\msrc} & \multicolumn{2}{c}{\mm} \\\hline
& & 0.3 & 0.7 & 0.3 & 0.7
& 0.3 & 0.7 & 0.3 & 0.7  \\ \hline\hline
\multirow{4}{.3cm}{\begin{sideways}\textbf{Late}\end{sideways}}\ldelim\{{4}{.1mm}
& \gmnemin & 0.150$\pm$0.004 & 0.071$\pm$0.008 & 0.146$\pm$0.005 & 0.165$\pm$0.012 & 0.664$\pm$0.004 & 0.644$\pm$0.004 & 0.567$\pm$0.005 & 0.549$\pm$0.005 \\
&\mces & 0.125$\pm$0.004 & 0.066$\pm$0.008 & 0.143$\pm$0.006 & 0.177$\pm$0.013 & 0.692$\pm$0.004 & 0.675$\pm$0.004 & 0.587$\pm$0.005 & 0.565$\pm$0.005 \\
&\mccs & 1.044$\pm$0.069 & 0.230$\pm$0.011 & 0.172$\pm$0.007 & 0.166$\pm$0.011 & 0.553$\pm$0.010 & 0.528$\pm$0.011 & 0.561$\pm$0.006 &